# Calculate harmonics of the baroclinic sea surface height data

In [1]:
!ls -lah /scratch/pawsey0219/ijanekovic/SSH/*201*.nc

-rw-r--r-- 1 ijanekovic pawsey0219 4.9G Nov 18 11:58 /scratch/pawsey0219/ijanekovic/SSH/roms_nws_20170101_12__qck.nc
-rw-r--r-- 1 ijanekovic pawsey0219 4.8G Nov 18 12:05 /scratch/pawsey0219/ijanekovic/SSH/roms_nws_20170111_13__qck.nc
-rw-r--r-- 1 ijanekovic pawsey0219 4.8G Nov 18 12:11 /scratch/pawsey0219/ijanekovic/SSH/roms_nws_20170121_13__qck.nc
-rw-r--r-- 1 ijanekovic pawsey0219 4.8G Nov 18 12:13 /scratch/pawsey0219/ijanekovic/SSH/roms_nws_20170131_13__qck.nc
-rw-r--r-- 1 ijanekovic pawsey0219 4.8G Nov 18 12:14 /scratch/pawsey0219/ijanekovic/SSH/roms_nws_20170210_13__qck.nc
-rw-r--r-- 1 ijanekovic pawsey0219 4.8G Nov 18 12:21 /scratch/pawsey0219/ijanekovic/SSH/roms_nws_20170220_13__qck.nc
-rw-r--r-- 1 ijanekovic pawsey0219 4.8G Nov 18 12:28 /scratch/pawsey0219/ijanekovic/SSH/roms_nws_20170302_13__qck.nc
-rw-r--r-- 1 ijanekovic pawsey0219 4.8G Nov 18 12:34 /scratch/pawsey0219/ijanekovic/SSH/roms_nws_20170312_13__qck.nc
-rw-r--r-- 1 ijanekovic pawsey0219 4.8G Nov 18 12:35 /scratch/pa

In [1]:
# Use a dask client
from dask.distributed import Client, LocalCluster, worker, progress, wait
import dask
from dask import delayed
from dask_jobqueue import SLURMCluster

In [2]:
cluster = SLURMCluster()
client=Client(cluster)
cluster.scale(40)
client


Bad key "backend.qt5" on line 5 in
/home/mrayson/.config/matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


Client Scheduler: tcp://146.118.38.118:40036 Dashboard: http://146.118.38.118:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
#from soda.dataio.suntans.sunpy import Spatial
import xarray as xr
import dask
from netCDF4 import Dataset
import numpy as np
from glob import glob
from datetime import datetime
import re

# from soda.dataio.suntans.sunxray import Sundask, Sunxray
# from soda.utils.myproj import MyProj
# from soda.utils.otherplot import axcolorbar
import soda.utils.harmonic_analysis as ha
from soda.utils.othertime import SecondsSince
# from soda.dataio.suntans.suntans_ugrid import ugrid

import matplotlib.animation as animation
from IPython.display import HTML

from matplotlib import rcParams

#rcParams['font.family'] = 'sans-serif'
#rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
#rcParams['font.serif'] = ['Bitstream Vera Sans']
rcParams["font.size"] = "14"
rcParams['axes.labelsize']='large'

import matplotlib.pyplot as plt

In [4]:
#basedir = '/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/'
#romsfiles = '{}/qck_*.nc'.format(basedir)

#basedir = '/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/verification'
#romsverfile = '{}/qck_ver_*.nc'.format(basedir)

romsverfile = '/scratch/pawsey0219/ijanekovic/SSH/*201*.nc'

#roms = xr.open_dataset(romsfile, chunks={})
def drop_all_coords(ds):
    return ds.reset_coords(drop=True)

roms = xr.open_mfdataset(sorted(glob(romsverfile)), concat_dim='ocean_time', 
                         parallel=True,)

# roms = xr.open_dataset(sorted(glob(romsverfile))[2], engine='netcdf4', chunks={})
# roms

# from netCDF4 import Dataset

# ds = Dataset(sorted(glob(romsverfile))[0])
# print(ds)
# ds.close()

In [5]:
roms

,Array,Chunk
Bytes,362.53 kB,3.86 kB
Shape,"(22658, 2)","(241, 2)"
Count,384 Tasks,96 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,181.26 kB,1.93 kB
Shape,"(22658, 2)","(241, 2)"
Count,384 Tasks,96 Chunks
Type,int32,numpy.ndarray
,Array,Chunk


In [21]:
# Let's try lazy loading all of the data and chunk 
varstring = 'northward velocity'
units = 'm s-1'
varname_out = 'v'
outfile = '../DATA/ROMS_NWS_2km_2017_2019_v_Harmonics.nc'

ssh = roms['v_sur_northward'].chunk({'ocean_time':-1, 'eta_rho':-1, 'xi_rho':1})
ssh

,Array,Chunk
Bytes,44.12 GB,43.68 MB
Shape,"(22658, 482, 1010)","(22658, 482, 1)"
Count,5131 Tasks,1010 Chunks
Type,float32,numpy.ndarray


In [22]:
# Compute the variance along the time axis
ssh_var = ssh.var(axis=0)
ssh_var = ssh_var.compute()

In [23]:
# plt.figure(figsize=(12,8))
# plt.pcolormesh(roms.lon_rho, roms.lat_rho, ssh_var, cmap='Reds', vmin=0)
# plt.colorbar()
# plt.gca().set_aspect('equal')

In [24]:
def build_lhs_dask(t,frq):
    """
    Construct matrix A
    """
    nt=t.shape[0]
    frq = np.array(frq)
    
    nf=frq.shape[0]
    nff=nf*2+1
    A=np.ones((nt,nff))
    for ff in range(0,nf):
        A[:,ff*2+1]=np.cos(frq[ff]*t)
        A[:,ff*2+2]=np.sin(frq[ff]*t)

    return dask.array.from_array(A, chunks=(A.shape[0],A.shape[1]))

def lstsq_dask(A,y):    
    """    
    Solve the least square problem

    Return:
        the complex amplitude 
        the mean
    """
    N=A.shape[1]
    sz = y.shape
    
    # Need to reshape the "y" array so that time is along the first dimension 
    # and all other dimensions are along the second 
    # (this probably uses up more memory than it should)
    b,res,rank,s = dask.array.linalg.lstsq(A, y.reshape(sz[0], np.prod(sz[1:])) )

    return b.reshape((N,)+sz[1:])


def harmonic_fit_dask(X, t, omega, A=None):
    
    if A is None:
        A = build_lhs_dask(t, omega)

    
    print('A: ',A.shape)
    print('X:',X.shape)
    
    # Remove NaNs
    X[dask.array.isnan(X)] = 0.

    # Least-squares matrix approach
    b = lstsq_dask(A, X) # This works on all columns of X!!
    print('b', b.shape)

    return b, A

twopi = 2*np.pi
tdaysec = 86400.

def nonstat_harmonic_fit_dask(X, t, omega, na, omega_A=twopi/(365*tdaysec), A=None):
    frq_all =[]
    for ff in omega:
        for n in range(-na,na+1):
            frq_all.append(ff+n*omega_A)

    Y, Alhs = harmonic_fit_dask(X, t, frq_all, A=A)
    
    # Break up the 
    aa = Y[0,...]
    Aa = Y[1::2,...]
    Ba = Y[2::2,...]
    
    return aa, Aa, Ba, frq_all, Alhs

In [25]:
reftime = datetime(1990,1,1)


In [26]:
# Do the harmonic fits
frq, names = ha.getTideFreq(['M2','S2','N2','K1','O1'])
time = SecondsSince(roms.ocean_time.values, basetime=reftime)
na = 3 # Number of annual harmonics to include
ii = 0

# We need to feed this function the dask.array and not the xarray.dataArray object
aa, Aa, Ba, frq_all,Alhs = nonstat_harmonic_fit_dask(ssh.data, time, frq, na)

# Alhs=None
# for ii in range(5):
#     Y = harmonic_fit_dask(ssh[:,:,ii], time, frq_all, A=Alhs)
# aa

A:  (22658, 71)
X: (22658, 482, 1010)
b (71, 482, 1010)


In [27]:
aa

,Array,Chunk
Bytes,3.89 MB,8.08 kB
Shape,"(482, 1010)","(1, 1010)"
Count,20850 Tasks,482 Chunks
Type,float64,numpy.ndarray


In [28]:
# Let's load up everything in memory
aa = aa.persist()
Aa = Aa.persist()
Ba = Ba.persist()

In [29]:
aa, Aa

(dask.array<getitem, shape=(482, 1010), dtype=float64, chunksize=(1, 1010), chunktype=numpy.ndarray>,
 dask.array<getitem, shape=(35, 482, 1010), dtype=float64, chunksize=(35, 1, 1010), chunktype=numpy.ndarray>)

In [30]:
# Create an xarray.Dataset and save
grdfile = '/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/grid.nc'
romsgrd = xr.open_dataset(grdfile)
romsgrd

<xarray.Dataset>
Dimensions:      (bath: 1, eta_psi: 481, eta_rho: 482, eta_u: 482, eta_v: 481, one: 1, xi_psi: 1009, xi_rho: 1010, xi_u: 1009, xi_v: 1010)
Coordinates:
    lat_rho      (eta_rho, xi_rho) float64 ...
    lon_rho      (eta_rho, xi_rho) float64 ...
    lat_u        (eta_u, xi_u) float64 ...
    lon_u        (eta_u, xi_u) float64 ...
    lat_v        (eta_v, xi_v) float64 ...
    lon_v        (eta_v, xi_v) float64 ...
Dimensions without coordinates: bath, eta_psi, eta_rho, eta_u, eta_v, one, xi_psi, xi_rho, xi_u, xi_v
Data variables:
    xl           (one) float64 ...
    el           (one) float64 ...
    JPRJ         |S2 ...
    spherical    (one) |S1 ...
    depthmin     (one) int16 ...
    depthmax     (one) int16 ...
    hraw         (bath, eta_rho, xi_rho) float64 ...
    h            (eta_rho, xi_rho) float64 ...
    f            (eta_rho, xi_rho) float64 ...
    pm           (eta_rho, xi_rho) float64 ...
    pn           (eta_rho, xi_rho) float64 ...
    dndx         (eta_rho, xi_rho) float64 ...
    dmde         (eta_rho, xi_rho) float64 ...
    x_rho        (eta_rho, xi_rho) float64 ...
    y_rho        (eta_rho, xi_rho) float64 ...
    x_psi        (eta_psi, xi_psi) float64 ...
    y_psi        (eta_psi, xi_psi) float64 ...
    x_u          (eta_u, xi_u) float64 ...
    y_u          (eta_u, xi_u) float64 ...
    x_v          (eta_v, xi_v) float64 ...
    y_v          (eta_v, xi_v) float64 ...
    lat_psi      (eta_psi, xi_psi) float64 ...
    lon_psi      (eta_psi, xi_psi) float64 ...
    mask_rho     (eta_rho, xi_rho) float64 ...
    mask_u       (eta_u, xi_u) float64 ...
    mask_v       (eta_v, xi_v) float64 ...
    mask_psi     (eta_psi, xi_psi) float64 ...
    angle        (eta_rho, xi_rho) float64 ...
    diff_factor  (eta_rho, xi_rho) float64 ...
    visc_factor  (eta_rho, xi_rho) float64 ...
    Tcline       float32 ...
    Vstretching  int32 ...
    Vtransform   int32 ...
    hc           float32 ...
    theta_b      float64 ...
    theta_s      float64 ...
    N            float64 ...
Attributes:
    type:                       GRID file
    gridid:                     theGridTitle
    history:                    Tue Nov 26 15:52:58 2019: ncks -v N /home/ivi...
    title:                      ROMS Application
    _NCProperties:              version=1|netcdflibversion=4.4.1.1|hdf5libver...
    author:                     Ivica Janekovic, ivica.janekovic@uwa.edu.au
    sponge:                     21-Nov-2019 created with /home/ivica/roms/Mat...
    NCO:                        4.4.2
    history_of_appended_files:  Wed Oct 24 20:14:44 2018: Appended file grid_...
    format:                     netCDF-4 with compression level 4 and short i...
    Conventions:                CF-1.4
    units:                      Variables are saved in millimeters (mm) for f...

In [31]:
# coords={'eta_rho':romsgrd.lon_rho, 'xi_rho':romsgrd.lat_rho}

In [32]:


SSH_var_da = xr.DataArray(ssh_var, dims=('eta_rho','xi_rho'),
                         attrs={'long_name':'sea surface height variance',
                               'units':'({})^2'.format(units)})
SSH_aa_da = xr.DataArray(aa, dims=('eta_rho','xi_rho'), 
                         attrs={'long_name':'{} mean amplitude'.format(varstring),
                               'units':units})

SSH_Aa_da = xr.DataArray(Aa, dims=('Ntide','eta_rho','xi_rho'),
                         attrs={'long_name':'{} real amplitude'.format(varstring),
                               'units':units})
SSH_Ba_da = xr.DataArray(Ba, dims=('Ntide','eta_rho','xi_rho'),
                         attrs={'long_name':'{} real amplitude'.format(varstring),
                               'units':units})

omega_da = xr.DataArray(frq_all, dims=('Ntide',), )

SSH_ds = xr.Dataset({'{}_var'.format(varname_out):SSH_var_da, \
                     '{}_aa'.format(varname_out):SSH_aa_da,\
                     '{}_Aa'.format(varname_out):SSH_Aa_da,'{}_Ba'.format(varname_out):SSH_Ba_da,
                    'omega':omega_da})
SSH_ds

,Array,Chunk
Bytes,3.89 MB,8.08 kB
Shape,"(482, 1010)","(1, 1010)"
Count,482 Tasks,482 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,136.31 MB,282.80 kB
Shape,"(35, 482, 1010)","(35, 1, 1010)"
Count,482 Tasks,482 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [33]:
SSH_merge = xr.merge([romsgrd, SSH_ds])
SSH_merge

,Array,Chunk
Bytes,3.89 MB,8.08 kB
Shape,"(482, 1010)","(1, 1010)"
Count,482 Tasks,482 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,136.31 MB,282.80 kB
Shape,"(35, 482, 1010)","(35, 1, 1010)"
Count,482 Tasks,482 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [34]:
SSH_merge.to_netcdf(outfile)
print('done')

done


In [35]:
print(outfile)

../DATA/ROMS_NWS_2km_2017_2019_v_Harmonics.nc
